# Calculate the reaction rates for the primary reactions, as well as the main secondary reactions

In [1]:
from volumetric_reaction_rates import *
from sigmav_functions import *
from power import *
from units_and_constants import *

# Calculate Startup Time and Net Power Loss

In [2]:
# Input Data
V_plasma = 150 * unit.m**3  # Plasma volume [m^3]
n_e_avg = 1.7e20 * unit.m**-3  # Average electron density [1/m^3]
n_D = n_e_avg  # Deuterium density, same as electron density initially [1/m^3]
Pf_DT = 1000 * unit.MW  # Fusion power for DT reactor [MW]
T_e_avg = 14 * unit.keV  # Average electron temperature [keV]
P_aux = 100 * unit.MW  # Auxiliary heating power [MW]
I_ST = 3 * unit.kg  # Inventory target [kg]

# PARAMETERS FOR THE FUSION REACTION
tau_p_T = 1 * unit.s  # Tritium confinement time [s]
tau_p_He3 = 1 * unit.s # He3 confinement time [s]

# PARAMETERS FOR THE TRITIUM PRODUCTION
TBR = 1.15  # Tritium breeding ratio for DT neutrons [-]
TBR_DDn = 1  # Tritium breeding ratio for DD neutrons [-]

# PARAMETERS FOR THE NET POWER LOSS
eta_th = 0.3  # Thermal efficiency of the reactor [-]
Q = 10  # Fusion gain factor [-]

In [3]:
# Calculate the reaction rates
DD_reaction_rates = volumetric_reaction_rates_DDfuel(n_D, T_e_avg, tau_p_T, tau_p_He3)
# Print the dictionary in the desired format with scientific notation and pretty units
print("Reaction Rates:")
for key, value in DD_reaction_rates.items():
    if hasattr(value, "magnitude"):  # Check if the value is a pint.Quantity
        print(f"{key}: {value:.2e~P}")
    else:
        print(f"{key}: {value:.2e}")

Reaction Rates:
rr_DDp: 1.75×10¹⁶ 1/m³/s
rr_DDn: 1.85×10¹⁶ 1/m³/s
rr_DT: 6.86×10¹⁴ 1/m³/s
rr_DHe3: 2.81×10¹² 1/m³/s
rr_tot: 3.67×10¹⁶ 1/m³/s
density_T: 1.68×10¹⁶ 1/m³
density_He3: 1.85×10¹⁶ 1/m³
prob_DDp: 4.76×10⁻¹
prob_DDn: 5.05×10⁻¹
prob_DT: 1.87×10⁻²
prob_DHe3: 7.66×10⁻⁵
prob_tot: 1.00×10⁰


## Estimate Tritium production

In [4]:
# calculate the tritium production rates
Tdot_fusion = DD_reaction_rates["rr_DDp"] * V_plasma - DD_reaction_rates["rr_DT"] * V_plasma # [1/s] rate of tritium production due to DDp fusions, considering the losses due to DT neutrons (NB. It is assumed that all the Tritium that is not burnt in DT fusios can be extracted from the system)
Tdot_breedingDT = TBR * DD_reaction_rates["rr_DT"] * V_plasma # [1/s] is the rate of tritium production due to DT neutrons interacting with the breeding blanket
Tdot_breedingDD = TBR_DDn * DD_reaction_rates["rr_DDn"] * V_plasma # [1/s] is the rate of tritium production due to DD neutrons interacting with the Li6 in the breeding blanket
Tdot_diff = V_plasma * DD_reaction_rates["density_T"] / tau_p_T # [1/s] is the rate of tritium production due to diffusion of tritium in the breeding blanket
Tdot_tot = Tdot_fusion + Tdot_breedingDT + Tdot_breedingDD # [1/s] is the total rate of tritium production in the system

"""
            |----> He3 + n (2.45 MeV)
            |            |--------------------> Tdot_breedingDD: tritium production due to DD neutrons interacting with the Li6 in the breeding blanket
    D + D ->|
      |     |----> T + p
      |            |
      |------------|---> D + T --> He4 + n (14.1 MeV)
                   |                     |----> Tdot_breedingDT: tritium production due to DT neutrons interacting with the breeding blanket
                   |
                   |--------------------------> Tdot_fusion: tritium production due to DDp fusions, considering the losses due to DT neutrons (NB. It is assumed that all the Tritium that is not burnt in DT fusios can be extracted from the system)
"""

print(f"tritium production rates:\n",
      f"    {Tdot_fusion:.2e~P} from the Tritium produced in the DDp channel and not burnt\n",
      f"    {Tdot_breedingDT:.2e~P} from the 14.1 MeV neutrons produced in the DT subchannel and bred (TBR = {TBR})\n",
      f"    {Tdot_breedingDD:.2e~P} from the 2.45 MeV neutrons produced in the DDn channel and bred (TBR_DDn = {TBR_DDn})\n",
      f"    --------------\n",
      f"    {Tdot_tot:.2e~P} total rate of tritium production in the system")


tritium production rates:
     2.52×10¹⁸ 1/s from the Tritium produced in the DDp channel and not burnt
     1.18×10¹⁷ 1/s from the 14.1 MeV neutrons produced in the DT subchannel and bred (TBR = 1.15)
     2.78×10¹⁸ 1/s from the 2.45 MeV neutrons produced in the DDn channel and bred (TBR_DDn = 1)
     --------------
     5.41×10¹⁸ 1/s total rate of tritium production in the system


## Startup time

In [5]:
# Calculate the startup time
N_ST = I_ST/molecular_weight_T.to("kg/mol")*N_A # [-] is the number of tritium atoms needed for startup (I_ST[g]/3.016[g/mol]*6.022e23[atoms/mol])
startup_time = N_ST / (Tdot_tot) # [s] is the time needed to produce the required amount of tritium for startup

# Print the startup time in different units using pint's conversion methods
print(f"Startup time: {startup_time:.2e~P} ({startup_time.to('hour'):.2e~P} = {startup_time.to('day'):.2e~P} = {startup_time.to('year'):.2e~P})")

Startup time: 1.11×10⁸ s (3.07×10⁴ h = 1.28×10³ d = 3.51×10⁰ a)


## Calculate Fusion Power

In [6]:
# Calculate the Reactor Power for DD reactions

Pf_DD = DD_reaction_rates["rr_DDp"]*E_DDp*V_plasma + DD_reaction_rates["rr_DDn"]*E_DDn*V_plasma # [W] is the power produced by DD reactions
Pf_DD_DT = DD_reaction_rates["rr_DT"]*E_DT*V_plasma # [W] is the power produced by DT sub-reactions
Pf_DD_DHe3 = DD_reaction_rates["rr_DHe3"]*E_DHe3*V_plasma # [W] is the power produced by DHe3 sub-reactions
Pf_DD_tot = Pf_DD.to('MW') + Pf_DD_DT.to('MW') + Pf_DD_DHe3.to('MW') # [W] is the total power produced in a DD reactor
print(f"Pf_DD_tot: {Pf_DD_tot:.2f~P} \n(of which {Pf_DD.to('MW'):.2f~P} is from DD reactions, {Pf_DD_DT.to('MW'):.2f~P} is from DT sub-reactions and {Pf_DD_DHe3.to('MW'):.2f~P} is from DHe3 sub-reactions)")

Pf_DT = fusion_power_50D50T(n_e_avg, T_e_avg, E_DT, V_plasma) # [W] is the power produced by DT reactions (the 1e-6 factor is needed to convert <sigmav> from cm^3/s to m^3/s)
# N.B. this is the total fusion power of a 50%D-50%T plasma in the selected conditions - it may not correspond to the expected 1000 MW power due to the use of average values of T and n_e
print(f"Pf_DT: {Pf_DT.to('MW'):.2f~P}\n(considering a 50%D-50%T plasma and using the formula (n_e_avg/2)^2 * <sigmav>_DT * E_DT * V_plasma)")


Pf_DD_tot: 3.52 MW 
(of which 3.23 MW is from DD reactions, 0.29 MW is from DT sub-reactions and 0.00 MW is from DHe3 sub-reactions)
Pf_DT: 735.10 MW
(considering a 50%D-50%T plasma and using the formula (n_e_avg/2)^2 * <sigmav>_DT * E_DT * V_plasma)


## Calculate net power loss

In [7]:
P_e_net_DD = calculate_P_e_net(Pf_DD, Q, eta_th) # [W] is the net electrical power produced by the reactor
P_e_net_DT = calculate_P_e_net(Pf_DT, Q, eta_th) # [W] is the net electrical power produced by the reactor
print(f"P_e_net_DD: {P_e_net_DD.to('MW'):.2f~P}")
print(f"P_e_net_DT: {P_e_net_DT.to('MW'):.2f~P}")

E_lost = np.abs(P_e_net_DD*startup_time - P_e_net_DT*startup_time) # [J] is the energy lost during the startup time operation in DD
print(f"E_lost: {E_lost.to('MJ'):.2e~P}")

Cost_per_MWh = 0.25/unit('MWh') # [USD] is the cost of electricity
Dollar_lost = Cost_per_MWh.to('1/J') * E_lost # [USD] is the cost of the lost energy during the startup time operation in DD
print(f"$ lost: {Dollar_lost:.2f~P} USD")

P_e_net_DD: 0.68 MW
P_e_net_DT: 153.70 MW
E_lost: 1.69×10¹⁰ MJ
$ lost: 1175640.53 USD
